# Trabajo Práctico: Agente para automatizar la búsqueda de repuestos

- **Curso:** DUIA - 2025, Módulo 6
- **Integrantes:** David Burckhardt, Martin Vazquez Arispe, Martin Caballero.
- **Objetivo:** Implementar un sistema inteligente que automatice la búsqueda, ranking y pedido de repuestos para una empresa distribuidora.

---
##  Índice del Notebook

1. **Consigna del trabajo**
2. **Configuracion de API Keys**

---

## 1. Consigna: agente(s) para automatizar la búsqueda de repuestos
- Dada una solicitud de repuestos específicos para una empresa distribuidora, un
agente debe identificar las especificaciones de dichos repuestos (según un
catálogo), a fin de poder buscarlos.
- El agente busca en primer lugar en el inventario de la empresa, y en en caso de
no encontrarlos (puede ser que encuentre solo algunos de ellos), debe consultar
en catálogos de proveedores.
- El sistema extrae información de las opciones encontradas, y genera un ranking
de alternativas, priorizando: 
    - Repuestos internos (si están disponibles).
    - Proveedores externos según criterios de optimización (por ej. costo-beneficio).
- Para repuestos internos: 
    - Se genera una orden de retiro del inventario y se notifica al almacén para su preparación. 
- Para repuestos externos: 
    - se envía un email automatizado al proveedor seleccionado para formalizar el pedido.

- Finalmente, se agenda la fecha estimada de entrega y detalles del pedido en el
sistema de seguimiento.
- Pueden incluirse pasos de "human in the loop" para verificar resultados antes de
tomar acciones

## 2. Dependencias necesarias

In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, Optional, Dict, List
from pydantic import BaseModel, Field
from langgraph.checkpoint.memory import MemorySaver
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import PydanticOutputParser
import json
from pymongo import MongoClient
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq



## 3. Configuración de API Keys y variables de entorno

Cargamos la `GROQ_API_KEY` desde el archivo `.env` e inicializamos el cliente LLM.

**Nota:** Asegúrate de tener un archivo `.env` en el directorio raíz con:
```
GROQ_API_KEY=tu_clave_aqui
```

In [2]:
load_dotenv()

# Verificar que la API key está configurada
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY no encontrada en .env")

# Inicializar el LLM de Groq
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.1,
    api_key=api_key
)

print("LLM de Groq inicializado correctamente")
print(f"   Modelo: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")


LLM de Groq inicializado correctamente
   Modelo: openai/gpt-oss-120b
   Temperature: 0.1


## 4. Conexión a Mongo DB

Cargamos la `MONGO_URI` desde el archivo `.env` e inicializamos.
IMPORTANTE: Contar con un usuario creado en la base de datos de Mongo. 

**Nota:** Asegúrate de tener un archivo `.env` en el directorio raíz con:
```
MONGO_URI=tu_clave_aqui
```

In [3]:
# Conectar a MongoDB Atlas
MONGO_URI = os.getenv("MONGO_URI")

if not MONGO_URI:
    raise ValueError("MONGO_URI no encontrada en .env")

client = MongoClient(MONGO_URI)
db = client.db_respuestos
collection = db.repuestos

print("✅ Conexión a MongoDB establecida")
print(f"   Base de datos: db_respuestos")
print(f"   Colección: repuestos")

✅ Conexión a MongoDB establecida
   Base de datos: db_respuestos
   Colección: repuestos


## 5. Definicion del Estado 

In [4]:
class ValidationRequest(BaseModel):
    is_parts_request: bool = Field(
        default=False,
        description="True si la consulta es una solicitud de repuestos o piezas. False si es una pregunta general o spam."
    )
    message: str = Field(
        default="",
        description="Mensaje del agente. Si es un pedido de repuestos, debe incluir los pasos siguientes. Indicar al cliente que debe se deben realizar consultas sobre repuestos."
    )

class ConversationResult(BaseModel):
    """
    Resultado del análisis conversacional.
    """
    enough_info: bool = Field(
        description="True si hay suficiente información para proceder con la búsqueda, False si se necesita más información"
    )
    message: str = Field(
        description="Mensaje para el usuario (pregunta si enough_info=False, confirmación si enough_info=True)"
    )

class ProductList(BaseModel):
    products: list[str] = Field(
        default=[],
        description="Es una lista con las descripciones que realizo el cliente de los productos que solicita"
    )

#Definimos el esquema mejorado
class AgentState(TypedDict):
    validation_result: ValidationRequest
    conversation_result: Optional[ConversationResult]  # Resultado del análisis conversacional
    messages: Annotated[list, add_messages]
    product_description: List[str]
    codigos_repuestos: Optional[List[str]]  # Lista de códigos (R-XXXX)
    info_completa: bool  # Si tenemos toda la información necesaria

    # Query optimizada para búsqueda semántica
    optimized_query: Optional[str]  # Query reformulada por el LLM

    # Resultados de búsqueda semántica
    semantic_results: Optional[List[Dict]]  # Candidatos de búsqueda semántica

    # Resultados de la búsqueda interna
    resultados_internos: Optional[Dict[str, List[Dict]]]  

    # Resultados de la búsqueda externa
    resultados_externos: Optional[Dict[str, List[Dict]]]

    # Análisis de disponibilidad por código
    disponibilidad: Optional[Dict[str, str]]  # {"R-0001": "full", "R-0002": "none"}
    codigos_para_externos: Optional[List[str]]

    # Reranking
    recomendaciones_llm: Optional[str]

## 6. Definicion de todas las Chains del LLM

### Chain de Extracción de respuestos

In [5]:
with open('prompts/identify_products.txt', 'r', encoding='utf-8') as f:
    IDENTIFY_PRODUCTS_PROMPT = f.read()

prompt = ChatPromptTemplate.from_messages([
    ("system", IDENTIFY_PRODUCTS_PROMPT),
    ("placeholder", "{messages}")
])

# Chain conversacional con respuesta estructurada
extraction_chain = prompt | llm.with_structured_output(ProductList)

### Chain de Conversación Inicial

In [6]:
with open('prompts/chat_inicial_prompt.txt', 'r', encoding='utf-8') as f:
    SYSTEM_PROMPT = f.read()

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("placeholder", "{messages}")
])

# Chain conversacional con respuesta estructurada
conversational_chain = prompt | llm.with_structured_output(ConversationResult)

### Chain de Validación de Intencion

In [7]:
with open('prompts/intention_classifier_prompt.txt', 'r', encoding='utf-8') as f:
    CLASSIFIER_SYSTEM_PROMPT = f.read()

validation_prompt = ChatPromptTemplate.from_messages([
    ("system", CLASSIFIER_SYSTEM_PROMPT),
    ("placeholder", "{messages}")
])

parser = PydanticOutputParser(pydantic_object=ValidationRequest)

validation_prompt_con_instrucciones = ChatPromptTemplate.from_messages(
    validation_prompt.messages + [
        ("human", "{format_instructions}")
    ]
).partial(
    format_instructions=parser.get_format_instructions()
)

validation_chain = (
    validation_prompt_con_instrucciones 
    | llm 
    | parser
)

### Chain de Reranking

In [8]:
with open('prompts/reranking_prompt.txt', 'r', encoding='utf-8') as f:
    RERANKING_PROMPT = f.read()

ranking_prompt_template = ChatPromptTemplate.from_messages([
    ("system", RERANKING_PROMPT),
    ("user", "Analiza las siguientes opciones y genera un ranking completo:\n\n{opciones_texto}")
])

ranking_chain = ranking_prompt_template | llm

### Chain de Requery Optimization


In [9]:
with open('prompts/requery_optimization_prompt.txt', 'r', encoding='utf-8') as f:
    REQUERY_PROMPT = f.read()

requery_prompt_template = ChatPromptTemplate.from_messages([
    ("system", REQUERY_PROMPT),
    ("user", "{user_message}")
])

requery_chain = requery_prompt_template | llm

## 7. Definición de los Nodos del Grafo

In [10]:
def extract_products_info(state: AgentState) -> AgentState:
    """
    Usa la chain de extracción estructurada para identificar todos los productos
    descritos por el usuario en los mensajes.
    """
    messages: list[BaseMessage] = state['messages']
    print(f"\n{'='*80}")
    print(f"📝 EXTRACCIÓN DE PRODUCTOS")
    
    # Solo invocamos con el último mensaje o la colección de mensajes
    try:
        # Usamos la cadena de extracción que retorna ProductList
        product_list_obj = extraction_chain.invoke({"messages": messages})
        
        # Convertir a la nueva estructura de seguimiento
        product_requests = []
        for product_name in product_list_obj.products:
            product_requests.append({
                "name": product_name,
                "info_needed": True, # Inicialmente se asume que se necesita más info
                "details": {},
                "info_solicitada": ["descripción detallada", "marca", "modelo/número de parte"] # Info de ejemplo
            })

        print(f"✅ Productos detectados: {len(product_requests)}")
        print(f"{'='*80}\n")
        
        return {
            "product_requests": product_requests,
            "product_description": product_list_obj.products # Mantenemos por si se usa en otra parte
        }
    
    except Exception as e:
        print(f"❌ Error en extracción de productos: {e}")
        # En caso de error, volvemos a solicitar información
        return {
            "product_requests": [],
            "messages": [AIMessage(content="❌ Lo siento, no pude identificar los repuestos que necesitas. ¿Podrías ser más específico?")]
        }

In [11]:
def classify_request(state: AgentState) -> AgentState:
    """
        Clasifica la solicitud del usuario.
    """

    print(f"\n{'='*80}")
    print(f"VALIDACION DE INTENCION")
    messages: list[BaseMessage] = state['messages']
    validation_result_object = validation_chain.invoke({"messages": messages})
    
    return {"validation_result": validation_result_object}

def set_val_message(state: AgentState) -> AgentState:
    """
        Establece el mensaje de validación.
    """
    return {"messages": [state['validation_result'].message]}

def route_classification(state: AgentState) -> str:
    """
        Ruta de clasificación.
    """
    if state['validation_result'].is_parts_request:
        print("✅ Mensaje valido")
        print(f"{'='*80}\n")
        return "continue"
    else:
        print("❌ Mensaje invalido")
        print(f"{'='*80}\n")
        return "end"

def check_product_info_completeness(state: AgentState) -> AgentState:
    """
    Verifica si se tiene suficiente información para proceder con la búsqueda
    para cada producto detectado. Genera un mensaje si falta info.
    """
    product_requests: List[Dict] = state.get('product_requests', [])
    messages: list[BaseMessage] = state['messages']
    
    print(f"\n{'='*80}")
    print(f"🧐 VERIFICACIÓN DE INFORMACIÓN POR PRODUCTO")
    
    missing_info_products = []
    
    # Lógica de verificación: En un entorno real, esta lógica sería más compleja
    # y podría involucrar otro LLM para parsear detalles del mensaje.
    
    # Por ahora, simulamos que falta información si el mensaje original es corto.
    # En un entorno educativo, puedes explicar que aquí se usaría un LLM más 
    # avanzado (o una función más inteligente) para analizar los detalles.
    
    # SIMULACIÓN (Reemplazar con lógica real de LLM/parser)
    # Si el estado 'info_completa' (global) es True, asumimos que ya se conversó.
    if state.get("info_completa", False):
        print("✅ Ya se había confirmado la información completa.")
        return {"info_completa": True}
        
    for product in product_requests:
        # Aquí iría la lógica avanzada: LLM para parsear product.name + mensajes
        # Por ahora, marcamos como necesitada
        if product.get("info_needed", True):
            missing_info_products.append(product)
            print(f"⚠️ {product['name']}: Falta información.")

    if not missing_info_products:
        print("✅ Información completa para todos los productos detectados.")
        return {"info_completa": True, "messages": [AIMessage(content="✅ Gracias, tenemos toda la información necesaria para iniciar la búsqueda de tus repuestos.")]}
    else:
        # Construir mensaje para solicitar la info específica por producto
        request_message = "Necesito más detalles para poder buscar los siguientes repuestos:\n"
        for i, product in enumerate(missing_info_products, 1):
            # En un entorno real, `product['info_solicitada']` se llenaría de
            # forma dinámica por el LLM en el nodo anterior.
            info_solicitada = product.get("info_solicitada", ["descripción detallada", "marca", "modelo/número de parte"])
            request_message += f"\n{i}. **{product['name']}**: Por favor, especifica: {', '.join(info_solicitada)}."
            
        print(f"❌ Falta información, solicitando: {len(missing_info_products)} productos.")
        return {
            "info_completa": False, 
            "messages": [AIMessage(content=request_message)]
        }

def route_after_extraction_check(state: AgentState) -> str:
    """
    Decide si continuar con optimización de query o pedir más info.
    """
    if state.get("info_completa", False):
        return "requery_optimization"  # Proceder a optimizar
    else:
        return "request_more_info"  # Volver a END para pedir nuevo input

def requery_optimization(state: AgentState) -> AgentState:
    """
    Optimiza la consulta del usuario para búsqueda semántica.
    ASUME que ya hay información suficiente.
    """
    messages = state["messages"]
    user_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
    
    # Concatenar TODOS los mensajes del usuario para contexto completo
    if user_messages:
        full_query = " ".join([msg.content for msg in user_messages])
    else:
        full_query = ""
    
    print(f"{'='*80}")
    print(f"🔄 OPTIMIZACIÓN DE QUERY")
    print(f"{'='*80}")
    print(f"📝 Query original: {full_query}")
    
    try:
        # Usar LLM para optimizar la query
        response = requery_chain.invoke({"user_message": full_query})
        optimized = response.content.strip()
        
        print(f"✅ Query optimizada: {optimized}")
        print(f"{'='*80}\n")
        
        # Query optimizada, agregar mensaje de confirmación
        mensaje_confirmacion = "✅ Perfecto, en un momento te traigo resultados..."
        
        return {
            "messages": [AIMessage(content=mensaje_confirmacion)],
            "optimized_query": optimized
        }
        
    except Exception as e:
        print(f"❌ Error en optimización de query: {e}\n")
        # Si falla, usar query original
        return {
            "messages": [AIMessage(content="✅ Perfecto, en un momento te traigo resultados...")],
            "optimized_query": full_query
        }

def semantic_search(state: AgentState) -> AgentState:
    """
    1. Toma la query optimizada del estado
    2. Genera embedding con sentence-transformers
    3. Busca similitud en MongoDB usando $vectorSearch
    4. Extrae los códigos R-XXXX de los documentos encontrados
    5. Retorna códigos para búsqueda interna/externa
    """
    from sentence_transformers import SentenceTransformer
    
    # Usar la query optimizada del estado anterior
    optimized_query = state.get("optimized_query", "")
    
    if not optimized_query:
        # Fallback: usar mensajes directos
        messages = state["messages"]
        user_queries = [msg.content for msg in messages if isinstance(msg, HumanMessage)]
        optimized_query = " ".join(user_queries)
    
    print(f"\n{'='*80}")
    print(f"🔍 BÚSQUEDA SEMÁNTICA")
    print(f"🧬 Query para embedding: {optimized_query}")
    print(f"{'='*80}\n")
    
    
    # Cargar modelo (384 dimensiones)
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
    # Generar embedding de la query optimizada
    query_embedding = model.encode(optimized_query).tolist()
    
    # Pipeline MongoDB Vector Search
    # Busca similitud en embedding_vector (que contiene: descripción + marca + categoría + modelo)
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index_repuestos",
                "path": "embedding_vector",
                "queryVector": query_embedding,
                "numCandidates": 100,
                "limit": 10
            }
        },
        {
            "$project": {
                "id_repuesto": 1,  # ← Extraemos el código (NO está en el embedding)
                "repuesto_descripcion": 1,
                "categoria": 1,
                "marca": 1,
                "modelo": 1,
                "score": {"$meta": "vectorSearchScore"}
            }
        }
    ]
    
    try:
        resultados = list(collection.aggregate(pipeline))
        
        if not resultados:
            mensaje = ("🤔 No encontré repuestos similares.\n\n"
                      "Ayúdame con más detalles:\n"
                      "- **Descripción**: ¿Qué tipo de repuesto? (rodamiento, filtro, bomba...)\n"
                      "- **Marca**: ¿Conoces el fabricante? (SKF, Bosch, FAG...)\n"
                      "- **Modelo**: ¿Tienes número de parte? (6204-2RS, HF35554...)\n"
                      "- **Categoría**: ¿De qué tipo es? (mecánico, eléctrico, neumático...)")
            return {
                "messages": [AIMessage(content=mensaje)],
                "info_completa": False
            }
        
        # Mostrar resultados encontrados
        print(f"📊 Encontrados {len(resultados)} resultados:\n")
        mensaje = "🔍 Repuestos encontrados por similitud:\n\n"
        
        codigos_encontrados = []
        
        for i, r in enumerate(resultados, 1):

            if '_id' in r:
                r['_id'] = str(r['_id'])
                
            codigo = r['id_repuesto']  # ← Código extraído del documento (no del embedding)
            descripcion = r['repuesto_descripcion']
            marca = r.get('marca', 'N/A')
            modelo = r.get('modelo', 'N/A')
            categoria = r.get('categoria', 'N/A')
            score = r.get('score', 0)
            
            print(f"{i}. {codigo}: {descripcion}")
            print(f"   Marca: {marca} | Modelo: {modelo} | Categoría: {categoria}")
            print(f"   Similitud: {score*100:.1f}%\n")
            
            mensaje += f"{i}. **{codigo}**: {descripcion}\n"
            mensaje += f"   📦 Marca: {marca} | 🔧 Modelo: {modelo}\n"
            mensaje += f"   📂 Categoría: {categoria} | 📊 Similitud: {score*100:.1f}%\n\n"
            
            # Agregar códigos con score > 50% (umbral ajustable)
            if score >= 0.50:
                codigos_encontrados.append(codigo)
        
        if codigos_encontrados:
            # Limitar a top 5 para no saturar
            codigos_top = codigos_encontrados[:5]
            mensaje += f"\n✅ Buscaré información detallada de: **{', '.join(codigos_top)}**"
            
            print(f"✅ Códigos seleccionados para búsqueda: {codigos_top}\n")
            print(f"{'='*80}\n")
            
            return {
                "messages": [AIMessage(content=mensaje)],
                "codigos_repuestos": codigos_top,
                "info_completa": True,
                "semantic_results": resultados
            }
        else:
            # Todos los resultados tienen score < 50%
            mensaje += ("\n⚠️ La similitud es baja. Los resultados podrían no ser exactos.\n"
                       "¿Puedes darme más detalles sobre la marca, modelo o especificaciones?")
            return {
                "messages": [AIMessage(content=mensaje)],
                "info_completa": False,
                "semantic_results": resultados
            }
            
    except Exception as e:
        print(f"❌ Error en búsqueda semántica: {e}")
        return {
            "messages": [AIMessage(content="❌ Error en la búsqueda. Por favor intenta nuevamente.")],
            "info_completa": False
        }

def route_after_semantic(state: AgentState) -> str:
    """
    Decide si continuar con búsqueda interna/externa o pedir más info.
    """
    info_completa = state.get("info_completa", False)
    codigos = state.get("codigos_repuestos")
    
    if info_completa and codigos and len(codigos) > 0:
        return "search_internal_parts"
    else:
        return "request_more_info"

def search_internal_parts(state: AgentState) -> AgentState:
    """
        Busca el repuesto en MongoDB usando el código identificado.
        Muestra los resultados por pantalla de forma detallada.
    """
    codigos = state.get("codigos_repuestos")
    try:
        # Buscar en MongoDB por id_repuesto
        resultados = list(collection.find({
            "id_repuesto": {"$in": codigos},
            "proveedor_tipo": "INTERNAL"
        }))

        resultados_por_codigo = {codigo: [] for codigo in codigos}
        for resultado in resultados:
            codigo = resultado.get("id_repuesto")
            if codigo in resultados_por_codigo:
                # Limpiar _id de MongoDB para serialización
                if '_id' in resultado:
                    resultado['_id'] = str(resultado['_id'])
                resultados_por_codigo[codigo].append(resultado)
        
                # Mostrar resultados básicos
        for codigo in codigos:
            opciones = resultados_por_codigo[codigo]
            if opciones:
                print(f"✅ {codigo}: {len(opciones)} proveedor(es) interno(s) encontrado(s)")
            else:
                print(f"❌ {codigo}: No encontrado en inventario interno")
        
        print(f"{'='*80}\n")
        
        return {"resultados_internos": resultados_por_codigo}

    except Exception as e:
        print(f"❌ Error al buscar en la base de datos: {e}")
        return {"resultados_internos": {codigo: [] for codigo in codigos}}

def check_stock(state: AgentState) -> AgentState:
    """
        Verifica la disponibilidad de los repuestos en el inventario interno.
        Muestra los resultados por pantalla de forma detallada.
    """
    codigos = state.get("codigos_repuestos")
    resultados_internos = state.get("resultados_internos", {})

    disponibilidad = {}
    necesita_externos = []

    for codigo in codigos:
        opciones_internas = resultados_internos.get(codigo, [])
        if not opciones_internas:
            disponibilidad[codigo] = "none"
            necesita_externos.append(codigo)
        else:
            total_stock = sum(opcion.get("stock_disponible", 0) for opcion in opciones_internas)
            if total_stock > 0:
                # Hay stock disponible
                disponibilidad[codigo] = "available_internal"
                print(f"✅ {codigo}: Disponible en inventario interno ({total_stock} unidades)")
                
                # Mostrar mejores opciones
                mejor_opcion = max(opciones_internas, key=lambda x: x.get("stock_disponible", 0))
                print(f"   └─ Mejor opción: {mejor_opcion.get('proveedor_nombre')} "
                      f"({mejor_opcion.get('stock_disponible')} unidades, "
                      f"{mejor_opcion.get('lead_time_dias')} días)")
            else:
                # Encontrado pero sin stock
                disponibilidad[codigo] = "no_stock_internal"
                necesita_externos.append(codigo)
                print(f"⚠️  {codigo}: Encontrado pero sin stock suficiente en el inventario interno")

    return {
        "disponibilidad": disponibilidad,
        "codigos_para_externos": necesita_externos  # ← Nueva clave en el estado
    }

def search_external_parts(state: AgentState) -> AgentState:
    """
    Busca en proveedores externos solo los repuestos que lo necesitan.
    """
    codigos_para_externos = state.get("codigos_para_externos", [])
    print(f"Buscando {len(codigos_para_externos)} repuesto(s): {', '.join(codigos_para_externos)}\n")
    
    try:
        # Búsqueda en batch
        resultados = list(collection.find({
            "id_repuesto": {"$in": codigos_para_externos},
            "proveedor_tipo": "EXTERNAL"
        }))
        
        # Organizar por código
        resultados_por_codigo = {codigo: [] for codigo in state.get("codigos_repuestos", [])}
        for resultado in resultados:
            codigo = resultado.get("id_repuesto")
            if codigo in resultados_por_codigo:
                if '_id' in resultado:
                    resultado['_id'] = str(resultado['_id'])
                resultados_por_codigo[codigo].append(resultado)
                print(f"✅ {codigo}: {len(resultados_por_codigo[codigo])} proveedor(es) externo(s) encontrado(s)")
        
        return {"resultados_externos": resultados_por_codigo}
        
    except Exception as e:
        print(f"❌ Error al buscar en proveedores externos: {str(e)}\n")
        return {"resultados_externos": {}}

def need_external_parts(state: AgentState) -> str:
    """
    Decide si se necesita buscar en proveedores externos.
    """
    codigos_para_externos = state.get("codigos_para_externos", [])
    
    if codigos_para_externos:
        return "search_external"
    else:
        return "reranking"

def generate_ranking(state: AgentState) -> AgentState:
    from utils import format_options_for_llm
    """
    Genera ranking usando SOLO el LLM.
    """
    codigos = state.get("codigos_repuestos", [])
    resultados_internos = state.get("resultados_internos", {})
    resultados_externos = state.get("resultados_externos", {})
    
    print(f"\n{'='*80}")
    print(f"🏆 GENERANDO RANKING")
    print(f"{'='*80}\n")
    
    # Preparar información para el LLM
    opciones_para_llm = []
    
    for codigo in codigos:
        internos = resultados_internos.get(codigo, [])
        externos = resultados_externos.get(codigo, [])
        
        # Combinar todas las opciones SIN ordenar ni scoring
        todas_opciones = []
        
        for opcion in internos:
            todas_opciones.append({**opcion, "tipo": "INTERNO"})
        for opcion in externos:
            todas_opciones.append({**opcion, "tipo": "EXTERNO"})
        if todas_opciones:
            # Formatear para el LLM
            opciones_para_llm.append(
                format_options_for_llm(codigo, todas_opciones)
            )
    
    if not opciones_para_llm:
        print("⚠️ No hay opciones para rankear\n")
        return {
            "ranking_por_codigo": {},
            "recomendaciones_llm": "No hay opciones disponibles."
        }
    
    # Crear el texto completo para el LLM
    opciones_texto = "\n\n".join(opciones_para_llm)
    
    try:
        # Invocar el LLM para que haga el ranking
        recomendaciones = ranking_chain.invoke({"opciones_texto": opciones_texto})
        recomendaciones_texto = recomendaciones.content
        print(recomendaciones_texto)
        
    except Exception as e:
        print(f"❌ Error al obtener ranking del LLM: {e}\n")
        recomendaciones_texto = "Error al generar ranking automático."
    
    return {
        "recomendaciones_llm": recomendaciones_texto
    }

## 8. Generación del Grafo

In [12]:
memory = MemorySaver()

#Defino el grafo
# Definimos el grafo
graph_builder = StateGraph(AgentState)

# Agrego los nodos al grafo
graph_builder.add_node("validation", classify_request)
graph_builder.add_node("set_val_message", set_val_message)
# Nodos para extracción y verificación
graph_builder.add_node("extract_products_info", extract_products_info) 
graph_builder.add_node("check_product_info_completeness", check_product_info_completeness) 
# El resto de los nodos...
graph_builder.add_node("requery_optimization", requery_optimization)
graph_builder.add_node("semantic_search", semantic_search)
graph_builder.add_node("search_internal_parts", search_internal_parts)
graph_builder.add_node("check_stock", check_stock)
graph_builder.add_node("search_external_parts", search_external_parts)
graph_builder.add_node("reranking", generate_ranking)

# Conecto los nodos
graph_builder.add_edge(START, "validation")

# Desde validation: si es pedido de repuestos → extract_products_info
graph_builder.add_conditional_edges(
    "validation",
    route_classification,
    {
        "continue": "extract_products_info",  # CAMBIO: Primero extraemos
        "end": "set_val_message"
    }
)

graph_builder.add_edge("set_val_message", END)

# Desde extracción → verificar
graph_builder.add_edge("extract_products_info", "check_product_info_completeness")

# Desde verificación: si tiene info suficiente → requery, si no → END (para nuevo input)
graph_builder.add_conditional_edges(
    "check_product_info_completeness",
    route_after_extraction_check,
    {
        "requery_optimization": "requery_optimization",
        "request_more_info": END  # Sale para que iniciar_agente pida nuevo input
    }
)

# Desde requery → semantic_search (siempre, ya que asumimos info completa)
graph_builder.add_edge("requery_optimization", "semantic_search")

# ... (El resto del flujo se mantiene igual)

graph_builder.add_conditional_edges(
    "semantic_search",
    route_after_semantic,
    {
        "search_internal_parts": "search_internal_parts",
        "request_more_info": END
    }
)

graph_builder.add_edge("search_internal_parts", "check_stock")
graph_builder.add_conditional_edges(
    "check_stock",
    need_external_parts,
    {
        "search_external": "search_external_parts",
        "reranking": "reranking"
    }
)
graph_builder.add_edge("search_external_parts", "reranking")
graph_builder.add_edge("reranking", END)

# Compilar sin interrupts
graph = graph_builder.compile(checkpointer=memory)

## 9. Inicialización del agente

In [13]:

def iniciar_agente(mensaje_usuario: str):
    config = {"configurable": {"thread_id": "1"}}
    
    # Estado inicial con todos los campos
    estado_inicial = {
        "messages": [HumanMessage(mensaje_usuario)],
        "validation_result": None,
        "conversation_result": None,  # ← NUEVO
        "codigos_repuestos": None,
        "info_completa": False,
        "optimized_query": None,
        "semantic_results": None,
        "resultados_internos": {},
        "resultados_externos": {},
        "disponibilidad": None,
        "codigos_para_externos": None,
        "recomendaciones_llm": None
    }
    
    result = graph.invoke(estado_inicial, config)
    
    # Loop de conversación
    while True:
        # Mostrar solo NUEVOS mensajes del agente
        mensajes_actuales = result.get("messages", [])
        print("🤖 Agente:",mensajes_actuales[-1].content)
        
        # Verificar si identificamos códigos de repuestos mediante búsqueda semántica
        if result.get("codigos_repuestos"):  # ← Simplificado: si hay códigos, terminar
            print(f"\n{'='*60}")
            print(f"📋 Códigos identificados: {result['codigos_repuestos']}")
            print(f"{'='*60}\n")
            print("\n✅ Búsqueda completada")
            break
        
        # Si no identificamos codigos de repuestos, pedir nuevo mensaje al usuario
        nuevo_mensaje = input("\n👤 Tú: ")
        
        if nuevo_mensaje.lower() in ["salir", "exit", "quit"]:
            print("\n👋 Conversación terminada")
            break
        
        # Volvemos a ejecutar el grafo con el nuevo mensaje
        nuevo_estado = {"messages": [HumanMessage(content=nuevo_mensaje)]}
        result = graph.invoke(nuevo_estado, config)
    
    return result

## 10. Inicio de Conversación

### Ejemplos de uso del agente

**Ejemplos de códigos de repuestos en la base de datos:**
- R-0001: Rodamiento rígido de bolas 6204 2RS
- R-0002: Filtro de aceite motor diésel
- R-0005: Bomba centrífuga 3 HP
- R-0010: Manómetro glicerina 0-16 bar

**Categorías disponibles:**
- RODAMIENTO, FILTRO, CORREA, SENSOR, BOMBA, ELECTRICO, NEUMATICA, MECANICO, INSTRUMENTO


In [ ]:
#main
print("="*60)
print("🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS")
print("="*60)
print("\nBienvenido al sistema de búsqueda de repuestos.")
print("El agente te ayudará a encontrar el repuesto que necesitas.")
print("\nPuedes escribir 'salir' en cualquier momento para terminar.\n")
print("-"*60)

#mensaje_usuario = input("\n👤 Tú: ")

#Para debugging
mensaje_usuario = "Necesito un Rodamiento rígido de bolas modelo 6204 2RS y dos Kits reparación válvula"

resultado = iniciar_agente(mensaje_usuario)

#Repuesto con el codigo R-0001
#Repuesto con codigo R-0005 y R-0002


🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS

Bienvenido al sistema de búsqueda de repuestos.
El agente te ayudará a encontrar el repuesto que necesitas.

Puedes escribir 'salir' en cualquier momento para terminar.

------------------------------------------------------------

VALIDACION DE INTENCION
